In [9]:
from IPython.display import HTML

HTML(r'''
<!DOCTYPE html><html><head><meta charset="UTF-8"><title>Radio Player</title>
<style>
body{font-family:Arial;background:#0d0d0d;color:#eee;display:flex;justify-content:center;padding-top:40px;transition:.3s;}
body.light{background:#f3f3f3;color:#222;}
#window{width:450px;border-radius:10px;background:#1a1a1a;box-shadow:0 0 20px #000;overflow:hidden;}
body.light #window{background:#fff;box-shadow:0 0 12px #aaa;}
#titleBar{background:#2a2a2a;color:#ddd;padding:10px;display:flex;align-items:center;justify-content:space-between;}
.titleBtn{width:18px;height:18px;margin-left:10px;cursor:pointer;border-radius:4px;background:#444;text-align:center;line-height:18px;}
.titleBtn:hover{background:#666;}
#hamburger{cursor:pointer;font-size:20px;margin-right:10px;}
#menuPanel{position:absolute;top:60px;width:180px;background:#222;padding:10px;border-radius:8px;display:none;z-index:999;}
body.light #menuPanel{background:#ddd;color:#111;}
#menuPanel div{padding:8px;cursor:pointer;}
#menuPanel div:hover{background:#444;}body.light #menuPanel div:hover{background:#bbb;}
#playerBox{text-align:center;padding:20px;}
#logo{width:180px;height:180px;object-fit:cover;border-radius:12px;background:#000;margin-bottom:20px;}
select{width:85%;padding:10px;background:#222;color:#eee;border-radius:6px;border:1px solid #444;}
body.light select{background:#eee;color:#111;border:1px solid #bbb;}
#controls{display:flex;justify-content:center;margin-top:20px;gap:25px;}
.iconBtn{cursor:pointer;font-size:28px;padding:8px;border-radius:8px;transition:.2s;}
.iconBtn:hover{transform:scale(1.15);}
#volumePopup{position:absolute;width:60px;background:#222;padding:10px;border-radius:10px;display:none;z-index:1000;}
body.light #volumePopup{background:#ddd;color:#111;}
#volumeSlider{writing-mode:bt-lr;transform:rotate(-90deg);width:120px;height:30px;}
#muteIcon{cursor:pointer;margin-top:10px;font-size:22px;}
#visualizer{width:100%;height:50px;background:#111;margin-top:30px;border-radius:8px;display:flex;align-items:flex-end;overflow:hidden;}
.bar{width:4px;margin:2px;background:lime;height:10px;border-radius:4px 4px 0 0;}
.modal{position:fixed;top:50%;left:50%;width:340px;transform:translate(-50%,-50%);background:#222;color:#eee;padding:20px;border-radius:12px;display:none;z-index:9999;}
body.light .modal{background:#fff;color:#000;}
audio::-webkit-media-controls-panel,audio::-webkit-media-controls-volume-slider,audio::-webkit-media-controls-mute-button{display:none!important;}
</style></head><body>

<div id="menuPanel"><div onclick="toggleTheme()">Toggle Theme</div><div onclick="openHelp()">Help</div><div onclick="openAbout()">About</div></div>

<div id="helpModal" class="modal"><h3>Help</h3><p>Use ▶ ⏸ ⏹ to control playback. Click 🔊 for volume popup.</p><button onclick="closeHelp()">Close</button></div>
<div id="aboutModal" class="modal"><h3>About</h3><p>Custom radio player with fallback logos + visualizer.</p><button onclick="closeAbout()">Close</button></div>

<div id="window"><div id="titleBar">
<div id="hamburger">☰</div><span>Radio Player</span>
<div style="display:flex;"><div class="titleBtn" onclick="minimizeWindow()">—</div><div class="titleBtn" onclick="closeWindow()">✕</div></div></div>

<div id="playerBox">
<img id="logo">
<div id="nowPlaying"><b>Now Playing:</b> None</div>

<select id="stationList">
<option data-logo="https://images.unsplash.com/photo-1520975918319-62bf67bf263b?w=600" value="https://stream.nightride.fm/chillsynth.mp3">Nightride Chillsynth</option>
<option data-logo="https://upload.wikimedia.org/wikipedia/commons/0/09/Violin_VL100.png" value="https://classical-stream.iowapublicradio.org/ClassicalLow.mp3">Classical – IPR</option>
<option data-logo="https://images.unsplash.com/photo-1526478806334-5fd488fcaabc?w=600" value="https://hirschmilch.de:7001/prog-house.mp3">Hirschmilch Prog House</option>
<option data-logo="https://upload.wikimedia.org/wikipedia/commons/3/32/Jazz.jpg" value="https://rozhlas.stream/jazz.mp3">Czech Radio Jazz</option>
<option data-logo="https://images.unsplash.com/photo-1511379938547-c1f69419868d?w=600" value="https://icecast.radiofrance.fr/fipelectro-midfi.mp3">FIP Electro</option>
</select>

<div id="controls">
<span class="iconBtn" onclick="playRadio()">▶</span>
<span class="iconBtn" onclick="pauseRadio()">⏸</span>
<span class="iconBtn" onclick="stopRadio()">⏹</span>
<span class="iconBtn" onclick="randomStation()">⏭</span>
<span class="iconBtn" onclick="toggleVolumePopup()">🔊</span>
</div>

<div id="volumePopup">
<input type="range" id="volumeSlider" min="0" max="1" step="0.01">
<div id="muteIcon" onclick="toggleMute()">🔊</div>
</div>

<div id="visualizer"></div>
<audio id="player" crossorigin="anonymous"></audio></div></div>

<script>
const player=document.getElementById("player"),stationList=document.getElementById("stationList"),
logo=document.getElementById("logo"),nowPlaying=document.getElementById("nowPlaying"),
menuPanel=document.getElementById("menuPanel"),helpModal=document.getElementById("helpModal"),
aboutModal=document.getElementById("aboutModal"),volumePopup=document.getElementById("volumePopup"),
volumeSlider=document.getElementById("volumeSlider"),muteIcon=document.getElementById("muteIcon");

let audioCtx,analyser,src;

/* MENU */
document.getElementById("hamburger").onclick=()=>menuPanel.style.display=menuPanel.style.display=="block"?"none":"block";
function toggleTheme(){document.body.classList.toggle("light");}
function openHelp(){helpModal.style.display="block";menuPanel.style.display="none";}
function closeHelp(){helpModal.style.display="none";}
function openAbout(){aboutModal.style.display="block";menuPanel.style.display="none";}
function closeAbout(){aboutModal.style.display="none";}

/* WINDOW */
function minimizeWindow(){let b=document.getElementById("playerBox");b.style.display=b.style.display=="none"?"block":"none";}
function closeWindow(){document.getElementById("window").style.display="none";}

/* FALLBACK IMAGES (Colab tested, 100% working) */
const fallback={
 synth:["https://images.unsplash.com/photo-1500530855697-b586d89ba3ee?w=600"],
 classical:["https://upload.wikimedia.org/wikipedia/commons/6/6d/Grand_piano.jpg"],
 house:["https://images.unsplash.com/photo-1518972559570-7cc1309f3229?w=600"],
 jazz:["https://upload.wikimedia.org/wikipedia/commons/7/7e/Jazz_band_at_the_Tivoli.jpg"],
 default:["https://images.unsplash.com/photo-1497032628192-86f99bcd76bc?w=600"]
};
function cat(n){
 n=n.toLowerCase();
 if(n.includes("synth"))return"synth";
 if(n.includes("classical"))return"classical";
 if(n.includes("house")||n.includes("electro"))return"house";
 if(n.includes("jazz"))return"jazz";
 return"default";
}
function fallbackLogo(){
 let name=stationList.options[stationList.selectedIndex].text,
 arr=fallback[cat(name)];
 logo.src=arr[0];
}
logo.onerror=fallbackLogo;

/* UPDATE DISPLAY */
function updateDisplay(){
 let o=stationList.options[stationList.selectedIndex];
 logo.src=o.dataset.logo;
 nowPlaying.innerHTML="<b>Now Playing:</b> "+o.text;
 setTimeout(()=>{if(logo.naturalWidth==0)fallbackLogo();},400);
}

/* AUDIO INIT */
function initAudio(){
 if(!audioCtx){
  audioCtx=new AudioContext();
  analyser=audioCtx.createAnalyser();
  analyser.fftSize=64;
  src=audioCtx.createMediaElementSource(player);
  src.connect(analyser);analyser.connect(audioCtx.destination);
 }
}

/* PLAYER CONTROLS */
function playRadio(){initAudio();player.src=stationList.value;updateDisplay();player.play();}
function pauseRadio(){player.pause();}
function stopRadio(){player.pause();player.src="";logo.src="";nowPlaying.innerHTML="<b>Now Playing:</b> None";}
function randomStation(){stationList.selectedIndex=Math.floor(Math.random()*stationList.length);playRadio();}
volumeSlider.oninput=()=>player.volume=volumeSlider.value;

/* VOLUME POPUP */
function toggleVolumePopup(){
 let r=event.target.getBoundingClientRect();
 volumePopup.style.left=r.left+"px";
 volumePopup.style.top=(r.top-150)+"px";
 volumePopup.style.display=volumePopup.style.display=="block"?"none":"block";
}
function toggleMute(){
 player.muted=!player.muted;
 muteIcon.innerText=player.muted?"🔇":"🔊";
}

/* VISUALIZER */
const bars=[],viz=document.getElementById("visualizer"),freq=new Uint8Array(32);
for(let i=0;i<32;i++){
 let d=document.createElement("div");d.classList.add("bar");viz.appendChild(d);bars.push(d);}
function animate(){
 requestAnimationFrame(animate);
 if(!analyser)return;
 analyser.getByteFrequencyData(freq);
 bars.forEach((b,i)=>b.style.height=(freq[i]/2)+"px");
}
animate();

updateDisplay();
</script></body></html>
''')
